In [18]:
from PIL import Image
import glob
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Activation, Flatten

In [52]:
img_dir_list = ['PIE/PIE/1/*.jpg', 'PIE/PIE/2/*.jpg', 'PIE/PIE/3/*.jpg', \
               'PIE/PIE/4/*.jpg', 'PIE/PIE/5/*.jpg', 'PIE/PIE/6/*.jpg', \
               'PIE/PIE/7/*.jpg', 'PIE/PIE/8/*.jpg', 'PIE/PIE/9/*.jpg', \
               'PIE/PIE/10/*.jpg', 'PIE/PIE/11/*.jpg', 'PIE/PIE/12/*.jpg', \
               'PIE/PIE/13/*.jpg', 'PIE/PIE/14/*.jpg', 'PIE/PIE/15/*.jpg', \
               'PIE/PIE/16/*.jpg', 'PIE/PIE/17/*.jpg', 'PIE/PIE/18/*.jpg', \
               'PIE/PIE/19/*.jpg', 'PIE/PIE/20/*.jpg']
image_list = []
image_own = []
train_list = []
train_own = []
test_list = []
test_own = []
image_label = []
own_label = [20] * 10

for i in range (0, 20):
    for filename in glob.glob(img_dir_list[i]):
        im = Image.open(filename)
        arr = np.array(im).flatten()
        image_list.append(arr)
        image_label.append(i)
for filename in glob.glob('own_image/*.jpg'):
    im = Image.open(filename)
    arr = np.array(im).flatten()
    image_own.append(arr)
    
image_list = np.asarray(image_list)
image_own = np.asarray(image_own)
image_label = np.asarray(image_label)
own_label = np.asarray(own_label)

train_list, test_list, train_label, test_label = train_test_split(image_list, image_label, test_size = 0.3)
train_list = np.asarray(train_list)
test_list = np.asarray(test_list)
train_label = np.asarray(train_label)
test_label = np.asarray(test_label)

train_own, test_own, train_own_label, test_own_label = train_test_split(image_own, own_label, test_size = 0.3)
train_own = np.asarray(train_own)
test_own = np.asarray(test_own)
train_own_label = np.asarray(train_own_label)
test_own_label = np.asarray(test_own_label)

random_indices = np.arange(0, train_list.shape[0])
np.random.shuffle(random_indices)
train_sample = train_list[random_indices[:500]]
train_sample_label = train_label[random_indices[:500]]
# print(train_sample)
train_sample = np.concatenate((train_sample, train_own))
train_sample_label = np.concatenate((train_sample_label, train_own_label))
test_list = np.concatenate((test_list, test_own))
test_label = np.concatenate((test_label, test_own_label))

train_sample = np.asarray(train_sample)
test_list = np.asarray(test_list)

# train_sample = train_sample.astype('int')
# test_list = test_list.astype('int')
train_sample = train_sample.reshape(train_sample.shape[0], 32, 32, 1)
test_list = test_list.reshape(test_list.shape[0], 32, 32, 1)
train_list = train_list.reshape(train_list.shape[0], 32, 32, 1)
# print(train_sample)
# print(train_own)

In [43]:
print(train_sample[0].shape)

(1024,)


In [53]:
test_label = to_categorical(test_label, num_classes = 21)
train_sample_label = to_categorical(train_sample_label, num_classes = 21)
train_label = to_categorical(train_label, num_classes = 21)

In [47]:
model = Sequential()

model.add(Conv2D(filters=20, kernel_size=5, strides=1, padding="same", input_shape=(32, 32, 1)))
model.add(MaxPool2D(pool_size=2, strides=2))

model.add(Conv2D(filters=50, kernel_size=5, strides=1, padding='same'))
model.add(MaxPool2D(pool_size=2, strides=2))

model.add(Flatten())
model.add(Dense(500))
model.add(Activation('relu'))

model.add(Dense(21))
model.add(Activation('softmax'))
adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 20)        520       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 20)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 16, 50)        25050     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 8, 8, 50)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 500)               1600500   
_________________________________________________________________
activation_7 (Activation)    (None, 500)              

In [50]:
model.fit(train_sample, train_sample_label, epochs=20, batch_size=32, validation_data=(test_list, test_label),
              shuffle=True, verbose=2, )

Train on 507 samples, validate on 1023 samples
Epoch 1/20
 - 1s - loss: 22.3582 - accuracy: 0.1045 - val_loss: 11.3841 - val_accuracy: 0.1554
Epoch 2/20
 - 1s - loss: 6.6868 - accuracy: 0.2564 - val_loss: 4.2734 - val_accuracy: 0.3363
Epoch 3/20
 - 1s - loss: 2.7691 - accuracy: 0.4635 - val_loss: 3.4274 - val_accuracy: 0.4154
Epoch 4/20
 - 1s - loss: 1.8245 - accuracy: 0.5996 - val_loss: 2.4003 - val_accuracy: 0.5347
Epoch 5/20
 - 1s - loss: 0.8150 - accuracy: 0.7535 - val_loss: 1.6622 - val_accuracy: 0.6706
Epoch 6/20
 - 1s - loss: 0.4860 - accuracy: 0.8600 - val_loss: 1.3522 - val_accuracy: 0.6989
Epoch 7/20
 - 1s - loss: 0.3354 - accuracy: 0.9112 - val_loss: 1.1830 - val_accuracy: 0.7292
Epoch 8/20
 - 1s - loss: 0.2132 - accuracy: 0.9389 - val_loss: 1.0364 - val_accuracy: 0.7605
Epoch 9/20
 - 1s - loss: 0.1234 - accuracy: 0.9822 - val_loss: 0.9865 - val_accuracy: 0.7830
Epoch 10/20
 - 1s - loss: 0.0811 - accuracy: 0.9882 - val_loss: 1.0264 - val_accuracy: 0.7742
Epoch 11/20
 - 1s - 

In [55]:
loss, accuracy = model.evaluate(train_sample, train_sample_label, verbose = 0)
print(accuracy * 100, loss)
loss,  accuracy = model.evaluate(test_list, test_label, verbose = 0)
print(accuracy* 100, loss)
loss,  accuracy = model.evaluate(train_list, train_label, verbose = 0)
print(accuracy* 100, loss)

85.60158014297485 0.6850768022048168
85.23949384689331 0.6508973632098876
85.21008491516113 0.696808374829653
